In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

import os
from scipy.sparse import coo_matrix

In [2]:
df = pd.read_csv(r"C:\Users\Sujan H G\OneDrive\Desktop\amazon.csv")
df.head(2)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...


In [3]:
data=df[["user_id","product_id","product_name","category","actual_price","rating","rating_count","about_product"]]

In [4]:
data.head(2)

,user_id,product_id,product_name,category,actual_price,rating,rating_count,about_product
0,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,"₹1,099",4.2,"24,269",High Compatibility : Compatible With iPhone 12...
1,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹349,4.0,"43,994","Compatible with all Type C enabled devices, be..."


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   user_id        1465 non-null   object
 1   product_id     1465 non-null   object
 2   product_name   1465 non-null   object
 3   category       1465 non-null   object
 4   actual_price   1465 non-null   object
 5   rating         1465 non-null   object
 6   rating_count   1463 non-null   object
 7   about_product  1465 non-null   object
dtypes: object(8)
memory usage: 91.7+ KB


In [6]:
data.isnull().sum()

user_id          0
product_id       0
product_name     0
category         0
actual_price     0
rating           0
rating_count     2
about_product    0
dtype: int64

In [7]:
data=data.dropna()

In [8]:
data.isnull().sum()

user_id          0
product_id       0
product_name     0
category         0
actual_price     0
rating           0
rating_count     0
about_product    0
dtype: int64

In [9]:
data.shape

(1463, 8)

In [10]:
data=data.drop_duplicates()

In [11]:
data.duplicated().sum()

0

In [12]:
data.shape

(1390, 8)

In [13]:
data['user_id']=data['user_id'].str.extract(r'(\d+)').astype(float)
data['product_id']=data['product_id'].str.extract(r'(\d+)').astype(float)

In [15]:
data.head(2)

,user_id,product_id,product_name,category,actual_price,rating,rating_count,about_product
0,3.0,7.0,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,"₹1,099",4.2,"24,269",High Compatibility : Compatible With iPhone 12...
1,3.0,98.0,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹349,4.0,"43,994","Compatible with all Type C enabled devices, be..."


In [21]:
num_user=data['user_id'].nunique()
num_items=data['product_id'].nunique()
num_rating=data['rating'].nunique()

print("Unique users:",num_user,"Unique Items:",num_items,"Number of unque rating:",num_rating)

Unique users: 88 Unique Items: 114 Number of unque rating: 28


#data preprocessing

In [23]:
data["features"]=data["product_name"]+" "+data["category"]+" "+data["about_product"]

In [27]:
tfid=TfidfVectorizer(stop_words='english')
tfid_matrix=tfid.fit_transform(data['features'])

Content Based Recommendation system

In [30]:
cosine_sim=linear_kernel(tfid_matrix,tfid_matrix)

In [34]:
cosine_sim

array([[1.        , 0.15515949, 0.32767569, ..., 0.02735086, 0.03610305,
        0.0020171 ],
       [0.15515949, 1.        , 0.17319734, ..., 0.        , 0.01172763,
        0.02225319],
       [0.32767569, 0.17319734, 1.        , ..., 0.01922905, 0.03348415,
        0.        ],
       ...,
       [0.02735086, 0.        , 0.01922905, ..., 1.        , 0.1458264 ,
        0.02038313],
       [0.03610305, 0.01172763, 0.03348415, ..., 0.1458264 , 1.        ,
        0.02967786],
       [0.0020171 , 0.02225319, 0.        , ..., 0.02038313, 0.02967786,
        1.        ]])

function to get recommendation

In [77]:
def get_recommendation(product_id,cosine_sim=cosine_sim):
    idx=data[data['product_id']==product_id].index[0]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores=sim_scores[1:11]
    prod_index=[i[0] for i in sim_scores]

    return data.iloc[prod_index][['product_id','product_name',"actual_price","rating"]]

In [78]:
data["product_id"]

0          7.0
1         98.0
2         96.0
3          8.0
4          8.0
         ...  
1460       8.0
1461       1.0
1462       9.0
1463       0.0
1464    1486.0
Name: product_id, Length: 1390, dtype: float64

In [79]:
recommendation=get_recommendation(8)

In [80]:
recommendation

,product_id,product_name,actual_price,rating
392,8.0,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,₹699,4.2
92,8.0,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,₹699,4.2
113,9.0,"boAt Type C A750 Stress Resistant, Tangle-free...",₹999,4.1
74,9.0,"boAt Type C A750 Stress Resistant, Tangle-free...",₹999,4.1
6,8.0,"boAt Micro USB 55 Tangle-free, Sturdy Micro US...",₹499,4.1
418,8.0,"boAt Micro USB 55 Tangle-free, Sturdy Micro US...",₹499,4.1
18,8.0,"boAt Type C A325 Tangle-free, Sturdy Type C Ca...",₹499,4.1
472,8.0,"boAt Type C A325 Tangle-free, Sturdy Type C Ca...",₹499,4.1
83,974.0,"boAt A 350 Type C Cable for Smartphone, Chargi...",₹799,4.4
245,8.0,Boat A 350 Type C Cable 1.5m(Jet Black),₹798,4.4


Collebrative recommendation system